# Setting up the Project
- import the necessary libaries
- load the json file (which is the DataForSEO API results)
- create dataframe

In [ ]:
# DEFINE BRAND INFORMATION
brand_name = "techcombank"
brand_domain = "techcombank.com"

In [46]:
import json
import pandas as pd


In [47]:
# load json file
with open('7. visrating/visrating.json', 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)  # Load the outer array

In [48]:
# convert to dataframe
df = pd.DataFrame(data)
df.columns = ['raw_data']
df['keyword'] = df['raw_data'].apply(lambda x: x['keyword'])
df = df.reindex(columns=['keyword', 'raw_data'])

# preview
display(df.head())
print(df.info())

,keyword,raw_data
0,xếp hạng tín nhiệm,"{'keyword': 'xếp hạng tín nhiệm', 'type': 'org..."
1,xếp hạng tín nhiệm rating,"{'keyword': 'xếp hạng tín nhiệm rating', 'type..."
2,dịch vụ xếp hạng tín nhiệm,"{'keyword': 'dịch vụ xếp hạng tín nhiệm', 'typ..."
3,chỉ số xếp hạng tín nhiệm,"{'keyword': 'chỉ số xếp hạng tín nhiệm', 'type..."
4,điểm xếp hạng tín nhiệm,"{'keyword': 'điểm xếp hạng tín nhiệm', 'type':..."


<class 'pandas.core.frame.DataFrame'>
Index: 113 entries, 0 to 112
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   113 non-null    object
 1   raw_data  113 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB
None


# Sub-data set that only includes keywords that activate AI Overviews
- filter the original dataset
- map the dataframe with these columns: ['keyword', 'items'], the 'items' column is the entire raw result of DataForSEO
- extract only the AI Overview element from 'items' (raw data)
- get the markdown ccontent of AI Overview to a seperate column
- get the references list of AI Overview in a seperate column

In [49]:
# create sub_dataset for ai_overview
aio_data = []
aio = df['raw_data'].tolist()

for json in aio:
    for i in json['items']:
        if 'ai_overview' in i['type']:
            aio_data.append(json)

df_aio = pd.DataFrame(aio_data)
df_aio = df_aio.drop(columns=['location_code','language_code', 'items_count','type', 'se_domain', 'check_url', 'datetime', 'spell', 'refinement_chips', 'se_results_count', 'item_types'])

# preview
display(df_aio.head())
print(df_aio.info())

,keyword,items
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab..."
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab..."
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   keyword  105 non-null    object
 1   items    105 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB
None


In [50]:
# get only ai_overview type element from items
df_aio['aio'] = df_aio['items'].apply(lambda x: [item for item in x if item['type'] == 'ai_overview'])

# preview
display(df_aio.head())
print(df_aio.info())

,keyword,items,aio
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran..."
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   keyword  105 non-null    object
 1   items    105 non-null    object
 2   aio      105 non-null    object
dtypes: object(3)
memory usage: 2.6+ KB
None


## Handle Markdown

In [51]:
# create column for aio_markdown
df_aio['aio_markdown'] = df_aio['aio'].apply(lambda x: x[0]['markdown'] if x else None)

display(df_aio.head())
print(df_aio.info())

,keyword,items,aio,aio_markdown
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của [quá tr...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   keyword       105 non-null    object
 1   items         105 non-null    object
 2   aio           105 non-null    object
 3   aio_markdown  104 non-null    object
dtypes: object(4)
memory usage: 3.4+ KB
None


In [52]:
# preview markdown
print(df_aio['aio_markdown'][0])

Xếp hạng tín nhiệm là quá trình đánh giá khả năng và mức độ sẵn sàng trả nợ của một tổ chức, doanh nghiệp, hoặc quốc gia.Kết quả xếp hạng được thể hiện bằng các ký hiệu (ví dụ: AAA, AA, A, BB, C), cho biết mức độ rủi ro tín dụng đối với người cho vay hoặc nhà đầu tư.Xếp hạng tín nhiệm là một công cụ quan trọng giúp các nhà đầu tư đánh giá rủi ro, đồng thời giúp các tổ chức nâng cao uy tín và thu hút vốn.[[1]](https://visrating.com/tin-tuc/xep-hang-tin-nhiem-la-gi.24.html#:~:text=T%E1%BB%95ng%20quan%20v%E1%BB%81%20c%C3%A1c%20s%E1%BA%A3n%20ph%E1%BA%A9m%20huy%20%C4%91%E1%BB%99ng%20v%E1%BB%91n%20c%E1%BB%A7a%20ng%C3%A2n%20h%C3%A0ng&text=X%E1%BA%BFp%20h%E1%BA%A1ng%20t%C3%ADn%20nhi%E1%BB%87m%20l%C3%A0%20m%E1%BB%99t%20ch%E1%BB%89%20s%E1%BB%91%20quan%20tr%E1%BB%8Dng,%C4%91%E1%BB%91i%20t%C3%A1c%20v%C3%A0%20kh%C3%A1ch%20h%C3%A0ng.)[[2]](https://saigonratings.com/dich-vu-xep-hang-tin-nhiem/#:~:text=l%C4%A9nh%20v%E1%BB%B1c%20n%C3%A0y.-,X%E1%BA%BFp%20h%E1%BA%A1ng%20t%C3%ADn%20nhi%E1%BB%87m%20l%C3%A0

In [53]:
# clean markdown citations with regex
import re


## Write function to clean markdown
def clean_markdown_citations(md: str) -> str:
    """
    - Remove citation-style links: [[label]](url)
    - Replace normal links with their visible text (exclude images)
    - Tidy whitespace created by removals
    """
    # 1) Drop citations like [[1]](http...) or [[abc]](http...)
    md = re.sub(r'\s*\[\[[^\[\]]+\]\]\s*\([^)]+\)', '', md)

    # 2) Turn [text](url) into "text", but keep images ![alt](url) intact
    md = re.sub(r'(?<!!)\[([^\]]+)\]\([^)]+\)', r'\1', md)

    # 2.1) Delete images ![alt](url)
    md = re.sub(r'!\[([^\]]*)\]\([^)]+\)', '', md)

    # 3) Tidy whitespace and spacing before punctuation
    md = re.sub(r'[ \t]+', ' ', md)
    md = re.sub(r'\s+([.,;:!?])', r'\1', md)

    # 4) Trim per-line and overall
    md = '\n'.join(line.strip() for line in md.splitlines())
    return md.strip()


## Pass markdown column through the function
df_aio['aio_markdown'] = df_aio['aio_markdown'].apply(lambda x: clean_markdown_citations(x) if x else None)
display(df_aio.head())
print(df_aio.info())
print(df_aio['aio_markdown'][0])

,keyword,items,aio,aio_markdown
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   keyword       105 non-null    object
 1   items         105 non-null    object
 2   aio           105 non-null    object
 3   aio_markdown  104 non-null    object
dtypes: object(4)
memory usage: 3.4+ KB
None
Xếp hạng tín nhiệm là quá trình đánh giá khả năng và mức độ sẵn sàng trả nợ của một tổ chức, doanh nghiệp, hoặc quốc gia.Kết quả xếp hạng được thể hiện bằng các ký hiệu (ví dụ: AAA, AA, A, BB, C), cho biết mức độ rủi ro tín dụng đối với người cho vay hoặc nhà đầu tư.Xếp hạng tín nhiệm là một công cụ quan trọng giúp các nhà đầu tư đánh giá rủi ro, đồng thời giúp các tổ chức nâng cao uy tín và thu hút vốn.

Mục đích của xếp hạng tín nhiệm:

- **Đánh giá rủi ro tín dụng:** Phản ánh khả năng vỡ nợ và mức độ tổn thất tài chính có thể xảy ra khi nghĩa vụ nợ đến hạn.
- **Hỗ trợ quyết định đầu tư:** 

## Handle references

In [54]:
# create column for aio_references
df_aio['aio_references'] = df_aio['aio'].apply(lambda x: x[0]['references'] if x else None)

display(df_aio.head())
print(df_aio.info())

,keyword,items,aio,aio_markdown,aio_references
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...,"[{'type': 'ai_overview_reference', 'position':..."
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...,"[{'type': 'ai_overview_reference', 'position':..."
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...,"[{'type': 'ai_overview_reference', 'position':..."
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...,"[{'type': 'ai_overview_reference', 'position':..."
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...,"[{'type': 'ai_overview_reference', 'position':..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   keyword         105 non-null    object
 1   items           105 non-null    object
 2   aio             105 non-null    object
 3   aio_markdown    104 non-null    object
 4   aio_references  104 non-null    object
dtypes: object(5)
memory usage: 4.2+ KB
None


In [55]:
# clean the dict keys
def clean_dict_keys(list_of_dicts: list) -> list:
    cleaned_list = []
    for idx, item in enumerate(list_of_dicts, start=1):
        # Create dictionary with keys in desired order
        cleaned_dict = {
            'rank': idx,
            'domain': item.get('domain', ''),
            'source': item.get('source', ''),
            'url': item.get('url', '')
        }
        cleaned_list.append(cleaned_dict)
    return cleaned_list

# apply the function to aio_references column
df_aio['aio_references'] = df_aio['aio_references'].apply(lambda x: clean_dict_keys(x) if x else None)

# preview aio_markdown
display(df_aio['aio_references'][5])
display(df_aio.head())
# df_aio['aio_references'][0]

[{'rank': 1,
  'domain': 'visrating.com',
  'source': 'VIS Rating',
  'url': 'https://visrating.com/tin-tuc/to-chuc-xep-hang-tin-nhiem-quoc-te.26.html'},
 {'rank': 2,
  'domain': 'vi.wikipedia.org',
  'source': 'Wikipedia',
  'url': 'https://vi.wikipedia.org/wiki/Fitch_Ratings#:~:text=*%20BB:%20d%E1%BB%85%20b%E1%BB%8B%20thay%20%C4%91%E1%BB%95i%20h%C6%A1n,NR:%20kh%C3%B4ng%20%C4%91%C6%B0%E1%BB%A3c%20x%E1%BA%BFp%20h%E1%BA%A1ng%20c%C3%B4ng%20khai.'},
 {'rank': 3,
  'domain': 'visrating.com',
  'source': 'VIS Rating',
  'url': 'https://visrating.com/tin-tuc/xep-hang-tin-nhiem-viet-nam.38.html#:~:text=%C4%90%C3%A2y%20l%C3%A0%20m%E1%BB%99t%20th%C6%B0%E1%BB%9Bc%20%C4%91o,tr%C6%B0%E1%BB%9Dng%20t%C3%A0i%20ch%C3%ADnh%20qu%E1%BB%91c%20t%E1%BA%BF.'},
 {'rank': 4,
  'domain': 'visrating.com',
  'source': 'VIS Rating',
  'url': 'https://visrating.com/chinh-sach/xem-online/ky-hieu-xep-hang-tin-nhiem-va-dinh-nghia.1#:~:text=5-,H%E1%BB%87%20Th%E1%BB%91ng%20B%E1%BA%ADc%20X%E1%BA%BFp%20H%E1%BA%A1ng%20T%C3

,keyword,items,aio,aio_markdown,aio_references
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour..."
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc..."
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...,"[{'rank': 1, 'domain': 'saigonratings.com', 's..."
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...,"[{'rank': 1, 'domain': 'baochinhphu.vn', 'sour..."
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour..."


In [56]:
# create column for number of references
df_aio['aio_references_count'] = df_aio['aio_references'].apply(lambda x: len(x) if x else 0)

display(df_aio.head())
print(df_aio.info())

,keyword,items,aio,aio_markdown,aio_references,aio_references_count
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",8
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",11
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...,"[{'rank': 1, 'domain': 'saigonratings.com', 's...",13
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...,"[{'rank': 1, 'domain': 'baochinhphu.vn', 'sour...",7
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               105 non-null    object
 1   items                 105 non-null    object
 2   aio                   105 non-null    object
 3   aio_markdown          104 non-null    object
 4   aio_references        104 non-null    object
 5   aio_references_count  105 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 5.1+ KB
None


## Export Data

In [57]:
# GET BRAND CITATION RANKING
def get_brand_rank(references: list) -> int:
    if references:
        for ref in references:
            if brand_domain in ref['domain'].lower() or brand_name in ref['source'].lower():
                return ref['rank']
    return None

df_aio[f"{brand_name}_rank"] = df_aio['aio_references'].apply(get_brand_rank)

display(df_aio.head(10))
print(df_aio.info())

,keyword,items,aio,aio_markdown,aio_references,aio_references_count,visrating_rank
0,xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",8,2.0
1,xếp hạng tín nhiệm rating,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",11,1.0
2,dịch vụ xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...,"[{'rank': 1, 'domain': 'saigonratings.com', 's...",13,4.0
3,chỉ số xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...,"[{'rank': 1, 'domain': 'baochinhphu.vn', 'sour...",7,2.0
4,điểm xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",11,2.0
5,thang xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Thang xếp hạng tín nhiệm là hệ thống các chữ c...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",9,1.0
6,danh sách xếp hạng tín nhiệm,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Danh sách xếp hạng tín nhiệm là bảng đánh giá ...,"[{'rank': 1, 'domain': 'vi.wikipedia.org', 'so...",9,6.0
7,xếp hạng tín nhiệm ảnh hưởng thế nào,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm ảnh hưởng lớn đến khả năng ...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",3,1.0
8,tác động của xếp hạng tín nhiệm,"[{'type': 'ai_overview', 'rank_group': 1, 'ran...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm có tác động lớn đến khả năn...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",8,1.0
9,xếp hạng tín nhiệm doanh nghiệp,"[{'type': 'organic', 'rank_group': 1, 'rank_ab...","[{'type': 'ai_overview', 'rank_group': 1, 'ran...",Xếp hạng tín nhiệm doanh nghiệp là đánh giá độ...,"[{'rank': 1, 'domain': 'tapchixaydung.vn', 'so...",11,4.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   keyword               105 non-null    object 
 1   items                 105 non-null    object 
 2   aio                   105 non-null    object 
 3   aio_markdown          104 non-null    object 
 4   aio_references        104 non-null    object 
 5   aio_references_count  105 non-null    int64  
 6   visrating_rank        90 non-null     float64
dtypes: float64(1), int64(1), object(5)
memory usage: 5.9+ KB
None


In [58]:
## remerge df_aio with df on keyword, the number of rows should stay the same as df (8031 keywords)
df_aio = df_aio.drop_duplicates('keyword')
df_merged = pd.merge(df, df_aio[['keyword', 'aio_markdown', 'aio_references', 'aio_references_count', f'{brand_name}_rank']], on='keyword', how='left')
display(df_merged.head())
print(df_merged.info())

## download to csv
df_download = df_merged.drop(columns=['aio_markdown', 'raw_data'])
# df_download = df_merged.drop(columns=['aio_markdown', 'type', 'se_domain', 'location_code', 'language_code','check_url','items','datetime','spell','refinement_chips','se_results_count','item_types'])

# remove source and url from aio_references for csv export
def simplify_refs(refs):
	# handle NaN (float), None, and non-iterable values safely
	if refs is None:
		return None
	# pandas uses float('nan') for missing values
	if isinstance(refs, float) and pd.isna(refs):
		return None
	# expected: list of dicts
	if isinstance(refs, list):
		return [{'rank': ref.get('rank'), 'domain': ref.get('domain')} for ref in refs]
	# if a single dict is present, convert it to list form
	if isinstance(refs, dict):
		return [{'rank': refs.get('rank'), 'domain': refs.get('domain')}]
	# fallback
	return None

df_download['aio_references'] = df_download['aio_references'].apply(simplify_refs)

df_download.to_csv('keywords.csv', index=False)


,keyword,raw_data,aio_markdown,aio_references,aio_references_count,visrating_rank
0,xếp hạng tín nhiệm,"{'keyword': 'xếp hạng tín nhiệm', 'type': 'org...",Xếp hạng tín nhiệm là quá trình đánh giá khả n...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",8.0,2.0
1,xếp hạng tín nhiệm rating,"{'keyword': 'xếp hạng tín nhiệm rating', 'type...",Xếp hạng tín nhiệm (rating) là đánh giá độc lậ...,"[{'rank': 1, 'domain': 'visrating.com', 'sourc...",11.0,1.0
2,dịch vụ xếp hạng tín nhiệm,"{'keyword': 'dịch vụ xếp hạng tín nhiệm', 'typ...",Dịch vụ xếp hạng tín nhiệm là dịch vụ phân tíc...,"[{'rank': 1, 'domain': 'saigonratings.com', 's...",13.0,4.0
3,chỉ số xếp hạng tín nhiệm,"{'keyword': 'chỉ số xếp hạng tín nhiệm', 'type...",Chỉ số xếp hạng tín nhiệm là thước đo định tín...,"[{'rank': 1, 'domain': 'baochinhphu.vn', 'sour...",7.0,2.0
4,điểm xếp hạng tín nhiệm,"{'keyword': 'điểm xếp hạng tín nhiệm', 'type':...",Điểm xếp hạng tín nhiệm là kết quả của quá trì...,"[{'rank': 1, 'domain': 'fiinratings.vn', 'sour...",11.0,2.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   keyword               113 non-null    object 
 1   raw_data              113 non-null    object 
 2   aio_markdown          104 non-null    object 
 3   aio_references        104 non-null    object 
 4   aio_references_count  105 non-null    float64
 5   visrating_rank        90 non-null     float64
dtypes: float64(2), object(4)
memory usage: 5.4+ KB
None


# COMPETITOR ANALYSIS
- it includes:
  - most cited sources
  - the relative average cited rank of each most cited source
  - cited probability of brand
  - average cited ranking position of brand

In [59]:
## get all the cited names

extracted_ref = df_aio['aio_references'].explode().reset_index(drop=True)
extracted_ref = pd.DataFrame(extracted_ref)
extracted_ref['domain'] = extracted_ref['aio_references'].apply(lambda x: x['domain'] if x else None)
extracted_ref['name'] = extracted_ref['aio_references'].apply(lambda x: x['source'] if x else None)
extracted_ref['rank'] = extracted_ref['aio_references'].apply(lambda x: x['rank'] if x else None)


display(extracted_ref.head(10))
print(extracted_ref.info())

## download raw citation data
# extracted_ref.to_csv('vna_citations_raw.csv', index=False)

,aio_references,domain,name,rank
0,"{'rank': 1, 'domain': 'fiinratings.vn', 'sourc...",fiinratings.vn,FiinRatings,1.0
1,"{'rank': 2, 'domain': 'visrating.com', 'source...",visrating.com,VIS Rating,2.0
2,"{'rank': 3, 'domain': 'visrating.com', 'source...",visrating.com,VIS Rating,3.0
3,"{'rank': 4, 'domain': 'saigonratings.com', 'so...",saigonratings.com,Saigon Ratings,4.0
4,"{'rank': 5, 'domain': 'visrating.com', 'source...",visrating.com,VIS Rating,5.0
5,"{'rank': 6, 'domain': 'saigonratings.com', 'so...",saigonratings.com,Saigon Ratings,6.0
6,"{'rank': 7, 'domain': 'visrating.com', 'source...",visrating.com,VIS Rating,7.0
7,"{'rank': 8, 'domain': 'www.youtube.com', 'sour...",www.youtube.com,YouTube ·,8.0
8,"{'rank': 1, 'domain': 'visrating.com', 'source...",visrating.com,VIS Rating,1.0
9,"{'rank': 2, 'domain': 'tapchicongthuong.vn', '...",tapchicongthuong.vn,Tạp chí Công Thương,2.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   aio_references  764 non-null    object 
 1   domain          764 non-null    object 
 2   name            764 non-null    object 
 3   rank            764 non-null    float64
dtypes: float64(1), object(3)
memory usage: 24.0+ KB
None


## turn into table of unique competitors
## How?
- export the brand names into a list, drop duplicates
- create a dataframe from the list
- create function that iterates through each brand of the new dataset and the old one

for each brand: 
    - create [domain] variable empty
    - create [rank] variable empty
    - iterates:
        - if brand name matches the 'source' in the old dataset:
            - append the 'domain' to [domain] variable
            - append the 'rank' to [rank] variable

then:
    - count number of cited times for each brand
    - get the unique domains of each brand
    - get the average rank of each brand

In [60]:
ref_brands = extracted_ref['name'].drop_duplicates().dropna().reset_index(drop=True)

ref_brands = ref_brands.to_list()
ref_brands = list(filter(None, ref_brands))

# print(ref_brands)

## create dataframe
ref_brands = pd.DataFrame(ref_brands)
ref_brands.columns = ['brand']

## write function
brand_list = []
for brand in ref_brands['brand']:
    brand_dict = {}
    brand_dict['brand'] = brand
    brand_dict['cited_count'] = extracted_ref[extracted_ref['name'] == brand].shape[0]
    brand_dict['unique_domains'] = extracted_ref[extracted_ref['name'] == brand]['domain'].unique().tolist()
    brand_dict['average_rank'] = extracted_ref[extracted_ref['name'] == brand]['rank'].mean()
    brand_dict['cited_probability'] = brand_dict['cited_count'] / extracted_ref.shape[0]
    ## create column for the number of prompts where the brand is cited
    brand_dict['cited_in_prompts'] = df_aio[df_aio['aio_references'].apply(lambda refs: any(ref and ref.get('source') == brand for ref in refs) if refs else False)].shape[0]
    brand_dict['prompt_cited_rate'] = brand_dict['cited_in_prompts'] / df_aio.shape[0]
    brand_list.append(brand_dict)
brand_list_df = pd.DataFrame(brand_list).sort_values(by='cited_count', ascending=False).reset_index(drop=True)

display(brand_list_df.head(20))


## download file
brand_list_df.to_csv('competitor.csv', index=False)
# display(ref_brands.head())
# print(ref_brands.info())

,brand,cited_count,unique_domains,average_rank,cited_probability,cited_in_prompts,prompt_cited_rate
0,VIS Rating,315,[visrating.com],4.768254,0.411765,90,0.857143
1,Saigon Ratings,98,[saigonratings.com],5.377551,0.128105,68,0.647619
2,FiinRatings,45,[fiinratings.vn],4.488889,0.058824,37,0.352381
3,Tạp chí Tài chính,26,[tapchitaichinh.vn],4.538462,0.033987,22,0.209524
4,S&I Ratings,22,[sniratings.com.vn],6.545455,0.028758,18,0.171429
5,VnEconomy,15,[vneconomy.vn],2.933333,0.019608,13,0.123810
6,baochinhphu.vn,13,[baochinhphu.vn],4.230769,0.016993,10,0.095238
7,Wikipedia,13,[vi.wikipedia.org],4.153846,0.016993,13,0.123810
8,Thư Viện Pháp Luật,11,[thuvienphapluat.vn],6.000000,0.014379,10,0.095238
9,investopedia.com,11,[translate.google.com],5.363636,0.014379,8,0.076190


In [61]:
## create interactive piechart for brand cited probability shares, only taking the top 10 brands
import plotly.express as px
top_10_brands = brand_list_df.head(10)
fig = px.pie(top_10_brands, values='cited_probability', names='brand', title='Top 10 Cited Brands Probability Share', hole=0.3)
fig.show()


In [62]:
## create interactive column chart for brand cited counts, only taking the top 20 brands
import plotly.express as px
top_20_brands = brand_list_df.head(20)
fig = px.bar(top_10_brands, x='brand', y='cited_count', title='Top 10 Cited Brands Count', text='cited_count')
fig.update_traces(textposition='outside')
fig.show()

## export charts to embeddable html
# fig.write_html('vna_top_10_cited_brands_probability_share.html')

# BRAND MENTION

In [63]:
# appand all competitors into a single list called brand_competitor, with only name and domain, name is "brand" in brand_list_df, domain is "unique_domains" in brand_list_df
brand_competitor = []
for index, row in brand_list_df.iterrows():
    brand_competitor.append({
        "name": row['brand'],
        "domain": row['unique_domains'] if row['unique_domains'] else None
    })

brand_competitor

[{'name': 'VIS Rating', 'domain': ['visrating.com']},
 {'name': 'Saigon Ratings', 'domain': ['saigonratings.com']},
 {'name': 'FiinRatings', 'domain': ['fiinratings.vn']},
 {'name': 'Tạp chí Tài chính', 'domain': ['tapchitaichinh.vn']},
 {'name': 'S&I Ratings', 'domain': ['sniratings.com.vn']},
 {'name': 'VnEconomy', 'domain': ['vneconomy.vn']},
 {'name': 'baochinhphu.vn', 'domain': ['baochinhphu.vn']},
 {'name': 'Wikipedia', 'domain': ['vi.wikipedia.org']},
 {'name': 'Thư Viện Pháp Luật', 'domain': ['thuvienphapluat.vn']},
 {'name': 'investopedia.com', 'domain': ['translate.google.com']},
 {'name': 'Tạp chí Xây dựng', 'domain': ['tapchixaydung.vn']},
 {'name': 'ASL LAW', 'domain': ['aslgate.com']},
 {'name': 'capital.com', 'domain': ['translate.google.com']},
 {'name': 'Báo Chính phủ', 'domain': ['baochinhphu.vn']},
 {'name': 'en.wikipedia.org', 'domain': ['translate.google.com']},
 {'name': 'ACB', 'domain': ['acb.com.vn']},
 {'name': 'Tạp chí Thị trường Tài chính Tiền tệ',
  'domain'

In [64]:
# Turn into dataframe
df_competitor = pd.DataFrame(brand_competitor)
df_competitor[:100]

,name,domain
0,VIS Rating,[visrating.com]
1,Saigon Ratings,[saigonratings.com]
2,FiinRatings,[fiinratings.vn]
3,Tạp chí Tài chính,[tapchitaichinh.vn]
4,S&I Ratings,[sniratings.com.vn]
...,...,...
78,Khoa Quản trị Kinh doanh,[kqtkd.duytan.edu.vn]
79,topi.vn,[topi.vn]
80,sacombank.com.vn,[www.sacombank.com.vn]
81,theleader.vn,[theleader.vn]


In [65]:
## get prompt citation rate for each competitor from brand_list_df using domain matching
# brand_list_df contains 'unique_domains' (list) and 'cited_in_prompts' (count)
from urllib.parse import urlparse
import re

def _extract_host(u: str) -> str:
    if not u:
        return ""
    u = str(u).strip().lower()
    # try parse URL first
    if "://" in u:
        try:
            host = urlparse(u).hostname or u
        except Exception:
            host = u
    else:
        host = u.split("/")[0].split(":")[0]
    # drop leading www.
    if host.startswith("www."):
        host = host[4:]
    return host

def _domain_label_match(comp: str, ud: str) -> bool:
    if not comp or not ud:
        return False
    comp = comp.lower().strip()
    ud_host = _extract_host(ud)
    # pattern ensures comp appears as a domain label (start, end or separated by dots)
    pattern = r'(^|\.)' + re.escape(comp) + r'($|\.)'
    return bool(re.search(pattern, ud_host))

## Checks brand mention for each competitor with regex in the aio_markdown column, use the "name" column and the "japanese_name" column of df_competitor if available
def check_brand_mention(brand: str, text: str, japanese_name: str = None) -> bool:
    if pd.isna(text) or not text:
        return False

    candidates = []
    if brand:
        candidates.append(str(brand))
    for name in candidates:
        # match whole word boundaries; escape special chars
        pattern = r'\b' + re.escape(name) + r'\b'
        if re.search(pattern, text, re.IGNORECASE):
            return True
    return False



# inherit cited_in_prompts, average_rank, prompt_cited_rate from brand_list_df
df_competitor = pd.merge(df_competitor, brand_list_df[['brand', 'cited_in_prompts', 'average_rank', 'prompt_cited_rate']], left_on='name', right_on='brand', how='left')
df_competitor = df_competitor.drop(columns=['brand'])
df_competitor['cited_in_prompts'] = df_competitor['cited_in_prompts'].fillna(0).astype(int)


df_competitor['mentioned'] = df_competitor.apply(
    lambda row: df_merged['aio_markdown']
        .apply(lambda text: check_brand_mention(row.get('name'), text, row.get('japanese_name')))
        .sum(),
    axis=1
)
df_competitor['prompt_cited_rate'] = df_competitor['cited_in_prompts'] / df_aio.shape[0]
df_competitor['mention_rate'] = df_competitor['mentioned'] / df_aio.shape[0]


display(df_competitor.sort_values(by='cited_in_prompts', ascending=False))
df_competitor.to_csv('brand-mention-summary.csv', index=False)

,name,domain,cited_in_prompts,average_rank,prompt_cited_rate,mentioned,mention_rate
0,VIS Rating,[visrating.com],90,4.768254,0.857143,29,0.276190
1,Saigon Ratings,[saigonratings.com],68,5.377551,0.647619,14,0.133333
2,FiinRatings,[fiinratings.vn],37,4.488889,0.352381,23,0.219048
3,Tạp chí Tài chính,[tapchitaichinh.vn],22,4.538462,0.209524,0,0.000000
4,S&I Ratings,[sniratings.com.vn],18,6.545455,0.171429,6,0.057143
...,...,...,...,...,...,...,...
78,Khoa Quản trị Kinh doanh,[kqtkd.duytan.edu.vn],1,2.000000,0.009524,0,0.000000
79,topi.vn,[topi.vn],1,2.000000,0.009524,0,0.000000
80,sacombank.com.vn,[www.sacombank.com.vn],1,4.000000,0.009524,0,0.000000
81,theleader.vn,[theleader.vn],1,4.000000,0.009524,1,0.009524


In [66]:
## Create plots for competitor mentions
import plotly.express as px
fig = px.bar(df_competitor.sort_values(by='mentioned', ascending=False), x='name', y='mentioned', title='Competitor Brand Mentions in AI Overview', text='mentioned')
fig.show()